In [26]:
import requests
import json

# Standard Data Science Helpers
import numpy as np
import pandas as pd
import scipy

import plotly as py
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode
#init_notebook_mode(connected=True)

import cufflinks as cf
cf.go_offline(connected=True)
cf.set_config_file(colorscale='plotly', world_readable=True)

# Extra options
pd.options.display.max_rows = 30
pd.options.display.max_columns = 25

# Show all code cells outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

import os
from IPython.display import Image, display, HTML

import ipywidgets as widgets
from ipywidgets import interact, interact_manual

import cv2
import matplotlib.pyplot as plt
import datetime as dt

%run login.py

loginquery = f"""
mutation {{
  logIn(
      email:\"{login}\",
      password:\"{pwd}\") {{
    jwt {{
      token
      exp
    }}
  }}
}}
"""

url = 'https://api.numina.co/graphql'
mylogin = requests.post(url, json={'query': loginquery})
token = mylogin.json()['data']['logIn']['jwt']['token']

devices = {'Streetscape': ('SWLSANDBOX1', 'img/Streetscape.png'), 'Outside': ('SWLSANDBOX3', 'img/Outside.png'),
           'UnderRaincoat': ('SWLSANDBOX2', 'img/UnderRaincoat.png')}

## Question 2

In [29]:
def getHeatmapImage(device, starttime, endtime, token=token):
    try:
        serial = devices[device][0]
    except KeyError:
        raise Exception('Invalid device name.')
    
    query = """
    query {
      feedHeatmaps(
        serialno: "%s",
        startTime:"%s",
        endTime:"%s",
        objClasses:["pedestrian"],
        timezone:"America/New_York") {
        edges {
          node {
            time
            objClass
            heatmap
          }
        }
      }
    }
    """ % (serial, starttime.strftime("%Y-%m-%dT%H:00:00"), endtime.strftime("%Y-%m-%dT%H:00:00"))

    heatdata = requests.post(url, json={'query': query}, headers = {'Authorization':token})
    
    heatarr = np.zeros((480,640))
    
    for point in heatdata.json()['data']['feedHeatmaps']['edges'][0]['node']['heatmap']:
        heatarr[point[1]][point[0]] = point[2]
    
    
    img = cv2.imread(devices[device][1])
    heatmap = cv2.resize(heatarr, (img.shape[1], img.shape[0]))
    heatmap = cv2.applyColorMap(np.uint8(255 * heatmap), cv2.COLORMAP_JET)
    combined_image = cv2.addWeighted(img,1,heatmap,0.3,0)
    
    return combined_image
    

In [30]:
cv2.imshow('x', getHeatmapImage('Outside', dt.datetime(2017, 5, 17, 0), dt.datetime(2019, 10, 17, 23)))
cv2.waitKey(0)

-1